In [ ]:
#!pip install matplotlib numpy pillow opencv-python --no-index
#!pip install scikit-learn tensorboard pytorch-lightning comet-ml --no-index
#!pip install --upgrade --force-reinstall torch torchvision pandas --no-index

## Imports and Utils

In [2]:
import torch
import os
import cv2
import random
import numpy as np

from PIL import Image
from glob import glob
from tqdm import tqdm, trange
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import json
import pytorch_lightning as pl
import sys

from lit_model import lit_gazetrack_model

from model import gazetrack_model
from gazetrack_data import gazetrack_dataset

from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from torchvision.transforms import Normalize, Resize, Compose, ToTensor, RandomCrop
root = os.environ['SLURM_TMPDIR']


In [4]:
import pandas as pd

In [5]:
f = '/home/abhinav1/Users/01046/images/'#Path of the User containing images and meta
weight_file = 'load_weights'
print(weight_file)
print(f, len(glob(f+'*.jpg')))

/home/abhinav1/projects/def-skrishna/beluga_backup/gaze-track/Checkpoints/GoogleCheckpoint_MITSplit.ckpt
/home/abhinav1/Users/01046/images/ 946


In [6]:
#Loading the model
model = gazetrack_model()
if(torch.cuda.is_available()):
    dev = torch.device('cuda:0')
else:
    dev = torch.device('cpu')
    
w = torch.load('/home/abhinav1/projects/def-skrishna/beluga_backup/gaze-track/Checkpoints/GoogleCheckpoint_MITSplit.ckpt',map_location=dev)['state_dict']
model.load_state_dict(w)
model.to(dev)
model.eval()

gazetrack_model(
  (eye_model): eye_model(
    (model): Sequential(
      (0): Conv2d(3, 32, kernel_size=(7, 7), stride=(2, 2))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01, inplace=True)
      (3): AvgPool2d(kernel_size=2, stride=2, padding=0)
      (4): Dropout(p=0.02, inplace=False)
      (5): Conv2d(32, 64, kernel_size=(5, 5), stride=(2, 2))
      (6): BatchNorm2d(64, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
      (7): LeakyReLU(negative_slope=0.01, inplace=True)
      (8): AvgPool2d(kernel_size=2, stride=2, padding=0)
      (9): Dropout(p=0.02, inplace=False)
      (10): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
      (11): BatchNorm2d(128, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
      (12): LeakyReLU(negative_slope=0.01, inplace=True)
      (13): AvgPool2d(kernel_size=2, stride=2, padding=0)
      (14): Dropout(p=0.02, inplace=False)
    )

In [7]:
# Add hook to [5] for pre ReLU and [6] for after ReLU
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook
model.combined_model[6].register_forward_hook(get_activation('out'))

In [37]:
data=[]
for j in os.listdir('/home/abhinav1/Users_MIT/02078/images'):#Iterating by specifying the User ID

    if(j.endswith('.jpg')):
        last4 = j[0:-4]
        current_images='/home/abhinav1/Users_MIT/02078/images'
        cur_meta='/home/abhinav1/Users_MIT/02078/meta'
        path=os.path.join(current_images,j)
        image = Image.open(path)
        w, h = image.size
        last4=last4+'.json'
        json_path=os.path.join(cur_meta,last4)
        #
        with open(json_path, 'r') as f:
              meta = json.load(f)
        screen_w, screen_h = meta['screen_w'], meta['screen_h']
        lx, ly, lw, lh = meta['leye_x'], meta['leye_y'], meta['leye_w'], meta['leye_h']
        rx, ry, rw, rh = meta['reye_x'], meta['reye_y'], meta['reye_w'], meta['reye_h']

        kps = [meta['leye_x1']/w, meta['leye_y1']/h, meta['leye_x2']/w, meta['leye_y2']/h, 
               meta['reye_x1']/w, meta['reye_y1']/h, meta['reye_x2']/w, meta['reye_y2']/h]

        l_eye = image.crop((max(0, lx), max(0, ly), max(0, lx+lw), max(0, ly+lh)))
        r_eye = image.crop((max(0, rx), max(0, ry), max(0, rx+rw), max(0, ry+rh)))

        l_eye = l_eye.transpose(Image.FLIP_LEFT_RIGHT)

        kps = torch.tensor(kps).float()

        out = torch.tensor([meta['dot_xcam'], meta['dot_y_cam']]).float()

        list_transforms = [Resize((128,128)),
                           ToTensor(),
                           Normalize(mean=(0.3741, 0.4076, 0.5425), std=(0.02, 0.02, 0.02)),]

        list_trfms = Compose(list_transforms)

        l_eye=list_trfms(l_eye)
        r_eye=list_trfms(r_eye)
        l_eye = l_eye[None]
        l_eye=l_eye.cuda()
        r_eye = r_eye[None]
        r_eye=r_eye.cuda()

        kps.shape
        kps=kps[None]
        kps.shape
        target=out
        target=target.cuda()
        target=target[None]
        kps=kps.cuda()
        with torch.no_grad():
            #pred = model(l_eye, r_eye, kps)
            pred_penultimate = list(model(l_eye, r_eye, kps).cpu().detach().numpy())
        pred_penultimate = list(activation["out"].detach().cpu().numpy())
        #pred=pred.cpu().detach().numpy()
        #pred=np.array(pred)
        pred_penultimate=np.array(pred_penultimate)
        target=target.cpu().detach().numpy()
        data.append([j,[pred_penultimate],[target]])

        #preds.extend(pred)


        #totals.extend(target.cpu().detach().numpy())
        #list_pred.append((j,pred))
        #target_pred.append((j,target))

In [38]:
data[0]

['02078__00446.jpg',
 [array([[1.4355229 , 3.08893   , 1.4311185 , 0.92534053]], dtype=float32)],
 [array([[  1.6516391, -10.521687 ]], dtype=float32)]]

In [39]:
import pandas as pd
df=pd.DataFrame(data,columns=['Image_ID','Penultimate_Output','GT_Value'])
path_csv = '/home/abhinav1/Users_MIT/CSVs_MIT_Dinesh/02078.csv'
df.to_csv(path_csv, index = False)

In [103]:
pred = list(model(l_eye, r_eye, kps).cpu().detach().numpy())

In [97]:
pred

[array([  1.0931695, -11.140883 ], dtype=float32)]

In [100]:
pred = list(activation["out"].detach().cpu().numpy())


In [101]:
pred

[array([1.0004677, 2.8215249, 2.4949894, 1.2041379], dtype=float32)]

In [102]:
out

tensor([  1.0640, -10.5840])

In [104]:
pred

[array([  1.0931695, -11.140883 ], dtype=float32)]